In [2]:
from rdflib import Graph, SKOS

In [6]:
EMMO = "http://emmo.info/emmo#"

ELUCIDATION = EMMO + "EMMO_967080e5_2f42_4eb2_a3a9_c58143e835f9"
ELECTROPEDIA_REF = EMMO + "EMMO_50c298c2_55a2_4068_b3ac_4e948c33181f"
IUPAC_REF = EMMO + "EMMO_fe015383_afb3_44a6_ae86_043628697aa2"
WIKIPEDIA_REF = EMMO + "EMMO_c84c6752_6d64_48cc_9500_e54a3c34898d"

# Define the URL of the turtle file containing the ontology
ontology_url = "https://raw.githubusercontent.com/emmo-repo/domain-electrochemistry/master/electrochemicalquantities.ttl"

# Markdown file with text
terms_md_path = "./terms.md"

In [41]:
def load_ttl_from_url(url:str)->Graph:
    g = Graph()
    g.parse(url, format="turtle")
    return g



def extract_terms_info_sparql(g: Graph)-> list:
    # Define a dictionary to hold the entities and their attributes
    text_entities = []

    # SPARQL QUERY #
    PREFIXES = """
        PREFIX emmo: <http://emmo.info/emmo#>
        PREFIX skos: <http://www.w3.org/2004/02/skos/core#> 
        """

    list_entity_types = ["IRI", "prefLabel", "Elucidation", "Alternative Label(s)", "IEC Reference", "IUPAC Reference", "Wikipedia Reference"]

    query =  PREFIXES  + """
        SELECT DISTINCT ?iri ?prefLabel ?elucidation ?altlabel ?iecref ?iupacref ?wikipediaref
        WHERE {
            ?iri skos:prefLabel ?prefLabel.

            OPTIONAL { ?iri emmo:EMMO_967080e5_2f42_4eb2_a3a9_c58143e835f9 ?elucidation . }
            OPTIONAL { ?iri skos:altLabel ?altLabel . }
            OPTIONAL { ?iri emmo:EMMO_50c298c2_55a2_4068_b3ac_4e948c33181f ?iecref . }
            OPTIONAL { ?iri emmo:EMMO_fe015383_afb3_44a6_ae86_043628697aa2 ?iupacref . }
            OPTIONAL { ?iri emmo:EMMO_c84c6752_6d64_48cc_9500_e54a3c34898d ?altLabel . }
        }
        """
        
    qres = g.query(query)

    for hit in qres:    
        hit_dict = {entity_type:str(entity) for entity_type, entity in zip(list_entity_types, hit)}
        text_entities.append(hit_dict)

    text_entities.sort(key=lambda e: e["prefLabel"])

    return text_entities



def generate_markdown(terms_md_path:str, entities_list:list):
    
    entites_md_text = ""


    for entity_dict in entities_list:

        entites_md_text += f"*** \n\n"

        pref_label = entity_dict["prefLabel"]
        entites_md_text += f"### {pref_label} \n\n"

        iri = entity_dict["IRI"]
        entites_md_text += f" [{iri}]({iri}) \n\n"

        elucidation = entity_dict["Elucidation"]
        if elucidation != "None":
            entites_md_text += f"{elucidation} \n\n"

        for annotation_key, annotation_value in entity_dict.items():
            if annotation_key not in ["IRI", "prefLabel", "Elucidation"]:
                if annotation_value != "None":
                    entites_md_text += f" *{annotation_key}*: {annotation_value} \n\n"


    with open(terms_md_path, "a", encoding="utf-8") as f:
        f.write(entites_md_text)

In [42]:
g = load_ttl_from_url(ontology_url)
entities_list = extract_terms_info_sparql(g)
print(entities_list)
generate_markdown(terms_md_path=terms_md_path, entities_list=entities_list)

[{'IRI': 'http://emmo.info/electrochemistry#electrochemistry_7fa406b0_512a_4d59_9e0c_5d8aba0103ae', 'prefLabel': 'ActivationOverpotential', 'Elucidation': 'Part of the electrode polarization arising from a charge-transfer step of the electrode reaction.', 'Alternative Label(s)': 'None', 'IEC Reference': 'https://www.electropedia.org/iev/iev.nsf/display?openform&ievref=482-03-05', 'IUPAC Reference': 'None', 'Wikipedia Reference': 'None'}, {'IRI': 'http://emmo.info/electrochemistry#electrochemistry_c955c089_6ee1_41a2_95fc_d534c5cfd3d5', 'prefLabel': 'ActiveMaterialLoading', 'Elucidation': 'Mass of active material in an electrode per unit electrode area.', 'Alternative Label(s)': 'None', 'IEC Reference': 'None', 'IUPAC Reference': 'None', 'Wikipedia Reference': 'None'}, {'IRI': 'http://emmo.info/electrochemistry#electrochemistry_214d925c_76c4_4f69_9afc_056a1ea82fc6', 'prefLabel': 'AdsorptionCurrent', 'Elucidation': 'Electric current that accompanies the adsorption of a species.', 'Alterna

In [50]:
PREFIXES = """
PREFIX emmo: <http://emmo.info/emmo#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
"""

query =  PREFIXES  + """
SELECT DISTINCT ?class ?prefLabel ?elucidation ?altlabel
WHERE {
    ?class skos:prefLabel ?prefLabel;
        rdfs:subClassOf <http://emmo.info/electrochemistry#electrochemistry_1cd1d777_e67b_47eb_81f1_edac35d9f2c6>;
        skos:altLabel ?altLabel.

    OPTIONAL { ?class emmo:EMMO_967080e5_2f42_4eb2_a3a9_c58143e835f9 ?elucidation . }  
}

GROUP BY ?class ?prefLabel ?elucidation
"""
qres = g.query(query)
print(list(qres))

[(rdflib.term.URIRef('http://emmo.info/electrochemistry#electrochemistry_03a6ce70_5085_4683_bb4e_fc3c18f7143a'), rdflib.term.Literal('OhmicOverpotential', lang='en'), rdflib.term.Literal('part of the electrode polarization arising from an electric current through an ohmic resistance within the electrode or the electrolyte.', lang='en'), None), (rdflib.term.URIRef('http://emmo.info/electrochemistry#electrochemistry_60741c58_a10d_4aa6_bb68_0066a6ff8e30'), rdflib.term.Literal('SurfaceOverpotential', lang='en'), rdflib.term.Literal('The potential of a working electrode relative to a reference electrode of the same kinds placed in the solution adjacent to the surface of the working electrode (just outside the double layer).', lang='en'), None), (rdflib.term.URIRef('http://emmo.info/electrochemistry#electrochemistry_7fa406b0_512a_4d59_9e0c_5d8aba0103ae'), rdflib.term.Literal('ActivationOverpotential', lang='en'), rdflib.term.Literal('Part of the electrode polarization arising from a charge-t

# Test to HTML

In [2]:
import os
import markdown
from bs4 import BeautifulSoup

# Read the Markdown file
with open('index.md', 'r') as file:
    markdown_text = file.read()

# Convert Markdown to HTML
html = markdown.markdown(markdown_text)

# Create a BeautifulSoup object
soup = BeautifulSoup(html, 'html.parser')

In [4]:
[tag.name for tag in soup.contents]

['h1',
 None,
 'p',
 None,
 'p',
 None,
 'p',
 None,
 'p',
 None,
 'p',
 None,
 'p',
 None,
 'h2',
 None,
 'p',
 None,
 'h3',
 None,
 'ul']